In [1]:
import numpy as np 
import pandas as pd 
import re
import plotly.graph_objects as go

In [2]:
exports = pd.read_csv("Exportacions.csv", sep=";", index_col=0)['2021']
imports = pd.read_csv("Importacions.csv", sep=";", index_col=0)['2021']
processed = pd.read_csv("Output_domèstic_processat.csv", sep=";", index_col=0)['2021']
extraction = pd.read_csv("Extracció_domèstica.csv", sep=";", index_col=0)['2021']

In [3]:
imports_primeres = imports.iloc[1:6]
imports_semimanufacturats = imports.iloc[7:12]
imports_manufacturats = imports.iloc[13:-1]

imports_primeres.index = imports_primeres.index.astype(str) + " (import primers)"
imports_semimanufacturats.index = imports_semimanufacturats.index.astype(str) + " (import semimanufacturats)"
imports_manufacturats.index = imports_manufacturats.index.astype(str) + " (import manufacturats)"

imports_primeres = imports_primeres.to_frame().reset_index(drop=False).rename({"2021":"value", "index":"source"}, axis=1)
imports_primeres.insert(1, "target", ["Matèries primeres (import)"]*len(imports_primeres))
imports_semimanufacturats = imports_semimanufacturats.to_frame().reset_index(drop=False).rename({"2021":"value", "index":"source"}, axis=1)
imports_semimanufacturats.insert(1, "target", ["Productes semimanufacturats (import)"]*len(imports_semimanufacturats))
imports_manufacturats = imports_manufacturats.to_frame().reset_index(drop=False).rename({"2021":"value", "index":"source"}, axis=1)
imports_manufacturats.insert(1, "target", ["Productes manufacturats (import)"]*len(imports_manufacturats))

In [4]:
imports_subcat = pd.concat([imports_primeres, imports_semimanufacturats, imports_manufacturats])
imports_subcat

,source,target,value
0,Matèries primeres (import primers),Matèries primeres (import),26442601
1,Biòtiques (import primers),Matèries primeres (import),14950670
2,Metàl·liques (import primers),Matèries primeres (import),73474
3,No metàl·liques (import primers),Matèries primeres (import),1735418
4,Combustibles fòssils (import primers),Matèries primeres (import),9683040
0,Biòtics (import semimanufacturats),Productes semimanufacturats (import),5906543
1,Metàl·lics (import semimanufacturats),Productes semimanufacturats (import),4894518
2,No metàl·lics (import semimanufacturats),Productes semimanufacturats (import),1567081
3,Combustibles fòssils (import semimanufacturats),Productes semimanufacturats (import),2000037
4,Altres productes semimanufacturats (import sem...,Productes semimanufacturats (import),1546831


In [5]:
imports_cat = imports_subcat.groupby('target')[['value']].sum().reset_index().rename({'target':'source'}, axis=1)
imports_cat = pd.concat([imports_cat, imports.to_frame().reset_index().rename({"2021":"value", "index":"source"}, axis=1).iloc[-1].to_frame().T])
imports_cat.insert(1, "target", "Imports")
imports_cat

,source,target,value
0,Matèries primeres (import),Imports,52885203
1,Productes manufacturats (import),Imports,43689426
2,Productes semimanufacturats (import),Imports,15915010
19,Altres importacions de l'exterior,Imports,5407608


In [6]:
imports_total = pd.Series({"source":"Imports", "target":"Catalonia", "value":imports_cat['value'].sum()}).to_frame().T
imports_total

,source,target,value
0,Imports,Catalonia,117897247


In [7]:
exports_primeres = exports.iloc[1:6]
exports_semimanufacturats = exports.iloc[7:12]
exports_manufacturats = exports.iloc[13:-1]

exports_primeres.index = exports_primeres.index.astype(str) + " (export primers)"
exports_semimanufacturats.index = exports_semimanufacturats.index.astype(str) + " (export semimanufacturats)"
exports_manufacturats.index = exports_manufacturats.index.astype(str) + " (export manufacturats)"

exports_primeres = exports_primeres.to_frame().reset_index(drop=False).rename({"2021":"value", "index":"target"}, axis=1)
exports_primeres.insert(1, "source", ["Matèries primeres (export)"]*len(exports_primeres))
exports_semimanufacturats = exports_semimanufacturats.to_frame().reset_index(drop=False).rename({"2021":"value", "index":"target"}, axis=1)
exports_semimanufacturats.insert(1, "source", ["Productes semimanufacturats (export)"]*len(exports_semimanufacturats))
exports_manufacturats = exports_manufacturats.to_frame().reset_index(drop=False).rename({"2021":"value", "index":"target"}, axis=1)
exports_manufacturats.insert(1, "source", ["Productes manufacturats (export)"]*len(exports_manufacturats))

In [8]:
exports_subcat = pd.concat([exports_primeres, exports_semimanufacturats, exports_manufacturats])
exports_subcat

,target,source,value
0,Matèries primeres (export primers),Matèries primeres (export),10047573
1,Biòtiques (export primers),Matèries primeres (export),5640688
2,Metàl·liques (export primers),Matèries primeres (export),123693
3,No metàl·liques (export primers),Matèries primeres (export),3523963
4,Combustibles fòssils (export primers),Matèries primeres (export),841739
0,Biòtics (export semimanufacturats),Productes semimanufacturats (export),6986128
1,Metàl·lics (export semimanufacturats),Productes semimanufacturats (export),2467339
2,No metàl·lics (export semimanufacturats),Productes semimanufacturats (export),3725972
3,Combustibles fòssils (export semimanufacturats),Productes semimanufacturats (export),1607478
4,Altres productes semimanufacturats (export sem...,Productes semimanufacturats (export),1607944


In [9]:
exports_cat = exports_subcat.groupby('source')[['value']].sum().reset_index().rename({'source':'target'}, axis=1)
exports_cat = pd.concat([exports_cat, exports.to_frame().reset_index().rename({"2021":"value", "index":"target"}, axis=1).iloc[-1].to_frame().T])
exports_cat.insert(1, "source", "Export")
exports_cat

,target,source,value
0,Matèries primeres (export),Export,20177656
1,Productes manufacturats (export),Export,45091864
2,Productes semimanufacturats (export),Export,16394861
19,Altres exportacions de l'exterior,Export,7318458


In [10]:
exports_total = pd.Series({"target":"Export", "source":"Catalonia", "value":exports_cat['value'].sum()}).to_frame().T
exports_total

,target,source,value
0,Export,Catalonia,88982839


In [11]:
processed["Altres emissions a l'atmosfera"] = processed[["Òxid de dinitrogen (N2O)", "Hidrofluorocarburs", "Perfluorocarburs", "Hexafluorur de sofre", "Contaminants orgànics persistents (COP)"]].astype(int).sum() + 102

processed.drop(["Total", "Abocament de residus sòlids", 
                "A partir de la combustió de biomassa", "Sense la combustió de biomassa",
                "Municipals. Al medi ambient", "Industrials. Al medi ambient", 
                "Metalls pesants", "Abocament de materials al mar", 
                "Adob orgànic (fems)", "Pesticides", 
                "Sal i altres materials anticongelants escampats per carreteres (incloent-hi graveta)", 
                "Dissolvents and gasos hilarants i altres", "Pèrdues dissipatives", 
                "Òxid de dinitrogen (N2O)", "Hidrofluorocarburs", "Perfluorocarburs", 
                "Hexafluorur de sofre", "Contaminants orgànics persistents (COP)", 
                "Emissions a l'atmosfera", "Emissions a l'aigua", "Ús dissipatiu dels productes"], inplace=True)

In [12]:
processed

Diòxid de carboni (CO2)                                                         37816053
Metà (CH4)                                                                        202998
Òxids nitrosos (NOx)                                                              169073
Monòxid de carboni (CO)                                                           146963
Compostos orgànics volàtils no metaners (NMVOC)                                   102369
Diòxid de sofre (SO2)                                                              20352
Amoníac (NH3)                                                                      77992
Partícules                                                                         91500
Altres emissions a l'atmosfera                                                      5692
Municipals. Als abocadors controlats                                             1373000
Industrials. Als abocadors controlats                                             615761
Eliminació de residus

In [13]:
processed_atmosfera = processed.iloc[0:9]
processed_residus_sòlids = processed.iloc[9:12]
processed_aigua = processed.iloc[12:15]
processed_dissipatiu = processed.iloc[15:]

processed_atmosfera = processed_atmosfera.to_frame().reset_index(drop=False).rename({"2021":"value", "index":"target"}, axis=1)
processed_atmosfera.insert(1, "source", ["Emissions a l'atmosfera"]*len(processed_atmosfera))
processed_residus_sòlids = processed_residus_sòlids.to_frame().reset_index(drop=False).rename({"2021":"value", "index":"target"}, axis=1)
processed_residus_sòlids.insert(1, "source", ["Abocament de residus sòlids"]*len(processed_residus_sòlids))
processed_aigua = processed_aigua.to_frame().reset_index(drop=False).rename({"2021":"value", "index":"target"}, axis=1)
processed_aigua.insert(1, "source", ["Emissions a l'aigua"]*len(processed_aigua))
processed_dissipatiu = processed_dissipatiu.to_frame().reset_index(drop=False).rename({"2021":"value", "index":"target"}, axis=1)
processed_dissipatiu.insert(1, "source", ["Ús dissipatiu dels productes"]*len(processed_dissipatiu))

In [14]:
processed_subcat = pd.concat([processed_atmosfera, processed_residus_sòlids, processed_aigua, processed_dissipatiu])
processed_subcat['value'] = processed_subcat['value'].astype(int)
processed_subcat

,target,source,value
0,Diòxid de carboni (CO2),Emissions a l'atmosfera,37816053
1,Metà (CH4),Emissions a l'atmosfera,202998
2,Òxids nitrosos (NOx),Emissions a l'atmosfera,169073
3,Monòxid de carboni (CO),Emissions a l'atmosfera,146963
4,Compostos orgànics volàtils no metaners (NMVOC),Emissions a l'atmosfera,102369
5,Diòxid de sofre (SO2),Emissions a l'atmosfera,20352
6,Amoníac (NH3),Emissions a l'atmosfera,77992
7,Partícules,Emissions a l'atmosfera,91500
8,Altres emissions a l'atmosfera,Emissions a l'atmosfera,5692
0,Municipals. Als abocadors controlats,Abocament de residus sòlids,1373000


In [15]:
processed_cat = processed_subcat.groupby('source')[['value']].sum().reset_index().rename({'source':'target'}, axis=1)
processed_cat.insert(1, "source", "Waste and Emissions")
processed_cat

,target,source,value
0,Abocament de residus sòlids,Waste and Emissions,4448761
1,Emissions a l'aigua,Waste and Emissions,197204
2,Emissions a l'atmosfera,Waste and Emissions,38632992
3,Ús dissipatiu dels productes,Waste and Emissions,579864


In [16]:
processed_total = pd.Series({"target":"Waste and Emissions", "source":"Catalonia", "value":processed_cat['value'].sum()}).to_frame().T
processed_total

,target,source,value
0,Waste and Emissions,Catalonia,43858821


In [17]:
extraction = extraction.drop(["Total", "Minerals metàl·lics", "ED per habitant", "ED per hectàrea", "ED/PIB a preus corrents (kg/€)"])
extraction.index = extraction.index.astype(str) + " (domestic extraction)"

In [18]:
extraction_cat = extraction.to_frame().reset_index()
extraction_cat.insert(1, "target", "Domestic extraction")
extraction_cat = extraction_cat.rename({"index":"source", "2021":"value"}, axis=1)
extraction_cat['value'] = extraction_cat['value'].astype(int)
extraction_cat 

,source,target,value
0,Biomassa (domestic extraction),Domestic extraction,8465715
1,Minerals no metàl·lics (domestic extraction),Domestic extraction,27139278
2,Combustibles fòssils (domestic extraction),Domestic extraction,4855


In [19]:
extraction_total = pd.Series({"source":"Domestic extraction", "target":"Catalonia", "value":extraction_cat['value'].sum()}).to_frame().T
extraction_total

,source,target,value
0,Domestic extraction,Catalonia,35609848


In [20]:
master = pd.concat([imports_subcat, imports_cat, imports_total, 
                    exports_subcat, exports_cat, exports_total, 
                    processed_subcat, processed_cat, processed_total, 
                    extraction_cat, extraction_total])

In [27]:
def clean_label(label):
    return re.sub(r"\s*\(.*?\)", "", label)

nodes = list(pd.unique(master[['source', 'target']].values.ravel('K')))

node_to_index = {node: i for i, node in enumerate(nodes)}

node_labels = [clean_label(node) for node in nodes]

master['source_id'] = master['source'].map(node_to_index)
master['target_id'] = master['target'].map(node_to_index)

In [36]:
colorlist = ["#fee8c8"]*5 + ["#fdbb84"]*5 + ["#e34a33"]*6 + ["#fee8c8"] + ["#e34a33"] + ["#fdbb84"] + ["#e34a33"] + ["#e34a33"] + \
    ["#ece7f2"] + ["#a6bddb"] + ["#2b8cbe"] + ["#2b8cbe"] + ["black"] + ["#e7e1ef"] + ["#ce1256"] + ["#d7b5d8"] + ["#df65b0"] + ["#ce1256"] +\
        ["#238443"]*4 + ["#ece7f2"]*5 + ["#a6bddb"]*5 + ["#2b8cbe"]*7 + ["#f1eef6"]*9 + ["#ce1256"]*3 + ["#d7b5d8"]*3 + ["#df65b0"]*4

In [28]:
nodes

['Matèries primeres (import primers)',
 'Biòtiques (import primers)',
 'Metàl·liques (import primers)',
 'No metàl·liques (import primers)',
 'Combustibles fòssils (import primers)',
 'Biòtics (import semimanufacturats)',
 'Metàl·lics (import semimanufacturats)',
 'No metàl·lics (import semimanufacturats)',
 'Combustibles fòssils (import semimanufacturats)',
 'Altres productes semimanufacturats (import semimanufacturats)',
 'Biòtics (import manufacturats)',
 'Metàl·lics (import manufacturats)',
 'No metàl·lics (import manufacturats)',
 'Combustibles fòssils (import manufacturats)',
 'Residus manufacturats (import manufacturats)',
 'Altres productes manufacturats (import manufacturats)',
 'Matèries primeres (import)',
 'Productes manufacturats (import)',
 'Productes semimanufacturats (import)',
 "Altres importacions de l'exterior",
 'Imports',
 'Matèries primeres (export)',
 'Productes semimanufacturats (export)',
 'Productes manufacturats (export)',
 'Export',
 'Catalonia',
 "Emissions

In [42]:
fig = go.Figure(data=[go.Sankey(
    arrangement="snap",
    valuesuffix = "tonnes",
    node=dict(
        pad=10,
        thickness=10,
        line=dict(color="black", width=0.5),
        label=node_labels,
        color=colorlist
    ),
    link=dict(
        source=master['source_id'], 
        target=master['target_id'],
        value=master['value']
    )
)])

fig.update_layout(
    title_text="Sankey Diagram of the Urban Metabolism of Catalonia", 
    font_size=10,
    width=1400, 
    height=600, 
    margin=dict(l=100, r=100, t=100, b=100)
)
fig.show()


In [43]:
import plotly.io as pio
pio.write_html(fig, file='index.html', auto_open=True)